# Importing libraries

In [1]:
import pandas as pd
import pandas
import datetime as dt
from datetime import datetime
from urllib.error import HTTPError
import pandas_market_calendars as mcal
import requests
from bs4 import BeautifulSoup
import re
import csv
import numpy as np
from sklearn import datasets, linear_model, metrics
import pickle

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy import stats
from sklearn.model_selection import KFold 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

# Unpickling the data

In [11]:
with open('exo_df.pickle', 'rb') as handle:
    exo_df = pickle.load(handle)

with open('cons_cyc_df_years.pickle', 'rb') as handle:
    cons_cyc_df_years = pickle.load(handle)


# Finding the best model for stock price

In [9]:
cmpny_model = {}
missing_revenue = []


def best_model(company_dict):
    kf = RepeatedKFold(n_splits=5, n_repeats=3, random_state=1)
    for keys in company_dict:
        company = company_dict[keys]
    
        X = company[company.columns[1:4]]#.values.reshape(-1,1)
        y = company[company.columns[0]]#.values.reshape(-1,1)
        try:
        #Linearregression
            model_lin = LinearRegression()
            Linear_reg_CV_score = np.mean(cross_val_score(model_lin, X, y, scoring='r2', cv=kf))
        
        #Ridgeregression
            #ridge = RidgeCV(cv=10).fit(X, y)
            #Ridge_reg_CV_score = ridge.score(X, y)
            model_ridge = Ridge()
            Ridge_reg_CV_score = np.mean(cross_val_score(model_ridge, X, y, scoring='r2', cv=kf))
        #Lassoregression
            #lasso = LassoCV(cv=10).fit(X, y)
            #Lasso_reg_CV_score = lasso.score(X, y)
            model_lasso = Lasso()
            Lasso_reg_CV_score = np.mean(cross_val_score(model_lasso, X, y, scoring='r2', cv=kf))
        
        #polynomialregression
            poly = PolynomialFeatures(degree=2, include_bias=True)
            x_train_trans = poly.fit_transform(X)
            PolyLinear_reg_CV_score  = np.mean(cross_val_score(model_lin, x_train_trans, y, cv=kf, scoring='r2'))
        
        except ValueError:
            missing_revenue.append(keys)
            print (missing_revenue)
        CV_scores = {'Linear Regression': Linear_reg_CV_score, 'Ridge Regression': Ridge_reg_CV_score,'Lasso Regression': Lasso_reg_CV_score, 'Polynomial Regression':PolyLinear_reg_CV_score}
    
        max_score = 0
        best_model = None
        for models in CV_scores:
            if (max_score < CV_scores[models]):
                max_score = CV_scores[models]
                best_model = models
        
        cmpny_model[keys] = (max_score, best_model)
      
    return cmpny_model
        

#best_model_dict = best_model(exo_df)


score_list_dict = {}
for keys in best_model_dict:
    if best_model_dict[keys][0] not in score_list_dict:
        score_list_dict[best_model_dict[keys][0]] = [keys]
    else:
        score_list_dict[best_model_dict[keys][0]].append(keys) 

final_model_dict = {}
cols = ['Company', 'Revenue(Million $)', 'PE Ratio', 'Interest Rate']

def nameToModelLookup(modelName):
    if method == 'Lasso Regression':
        return Lasso()
    if method == 'Ridge Regression':
        return Ridge()
    return None

for keys in sorted(score_list_dict.keys(), reverse = True)[:25]:
    company, method = ((score_list_dict[keys][0]), best_model_dict[score_list_dict[keys][0]][1])
    X = exo_df[company][exo_df[company].columns[1:4]]
    y = exo_df[company][exo_df[company].columns[0]]
    if nameToModelLookup(method):
        if not method in final_model_dict:
            final_model_dict[method] = pd.DataFrame(columns = cols)
        model = nameToModelLookup(method)
        model.fit(X,y)
        coef_val = model.coef_
        final_model_dict[method] = final_model_dict[method].append({'Company':company,'Revenue(Million $)': coef_val[0],
                                                'PE Ratio' : coef_val[1], 'Interest Rate': coef_val[2]}, ignore_index = True)
    
            
final_model_dict 

    

    

    

    
    

  
    
    



/usr/local/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
15 fits failed out of a total of 15.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_base.py", line 662, in fit
    X, y = self._validate_data(
  File "/usr/local/lib/python3.9/site-packages/sklearn/base.py", line 576, in _validate_data
    X, y = check_X_y(X, y, **check_params)
  File "/usr/local/lib/python3.9/site-packages/sklearn

['IHG']


/usr/local/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:771: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 762, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 103, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
  File "/usr/local/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 258, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "/usr/local/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
  File "/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_base.py", line 362, in predict
    return self._decision_function(X)
 

['IHG', 'NLS']


{'Lasso Regression':   Company  Revenue(Million $)  PE Ratio  Interest Rate
 0    ROST            0.023282  1.670204       0.000000
 1    AMZN            0.020295  0.004631       7.015890
 2    GNTX            0.039223  0.000337       0.000000
 3    SBUX            0.015357  0.074680       0.840102
 4    EXPE            0.062527 -0.013162       0.082457
 5    TSCO            0.044255  2.406112      -0.000000
 6    LCII            0.208205 -0.032503       1.390622,
 'Ridge Regression':   Company  Revenue(Million $)  PE Ratio  Interest Rate
 0    TXRH            0.092241  0.592552       1.237912
 1    PZZA            0.311858  0.518361       1.259505
 2    MNRO            0.187665  1.330326      -0.790139
 3     AAP            0.016050  7.480063      -6.401526}

# one, five and ten years high momentum stock list

In [12]:
hqm_ind = ['ticker',
        "one year price return",
        "one year return percentile",
        "five year price return",
        "five year return percentile",
        "ten year price return",
        "ten year return percentile",
          ]

year_list = ['one', 'five', 'ten']
hqm_score = pd.DataFrame(columns = hqm_ind)
for keys in cons_cyc_df_years:
    high_momntm_df = cons_cyc_df_years[keys].loc[:,(cons_cyc_df_years[keys].columns == 'Close')]
    one_year_return = high_momntm_df.asfreq('1Y', method = 'ffill').pct_change().dropna().mean().squeeze()
    five_year_return = high_momntm_df.asfreq('5Y', method = 'ffill').pct_change().dropna().mean().squeeze()
    ten_year_return = high_momntm_df.asfreq('10Y', method = 'ffill').pct_change().dropna().mean().squeeze()
    hqm_score = hqm_score.append({'ticker':keys, "one year price return":one_year_return,"one year return percentile": None, "five year price return": five_year_return, "five year return percentile" :None,"ten year price return":ten_year_return, "ten year return percentile":None}, ignore_index = True)


for rows in hqm_score.index:
    for year in year_list:
        hqm_score.loc[rows,'{} year return percentile'.format(year)] = stats.percentileofscore(hqm_score['{} year price return'.format(year)], 
                                                                              hqm_score.loc[rows,'{} year price return'.format(year)]) 

#alternate way to calculate percentile   
#hqm_score['one year return percentile'] = hqm_score['one year price return'].rank(pct = True)



hqm_score

,ticker,one year price return,one year return percentile,five year price return,five year return percentile,ten year price return,ten year return percentile
0,AAN,0.082398,19.607843,0.304979,33.333333,0.695209,47.058824
1,ANF,-0.051728,2.614379,-0.505350,0.653595,-0.800787,1.960784
2,AAP,0.193770,64.705882,1.210390,78.431373,3.825812,89.542484
3,AIN,0.124032,35.294118,0.462518,39.869281,0.585392,43.137255
4,AMZN,0.492866,97.385621,3.359363,99.346405,18.003294,100.0
...,...,...,...,...,...,...,...
148,URBN,0.107913,28.104575,0.115041,18.954248,0.236648,32.679739
149,MTN,0.197819,67.320261,1.561267,85.620915,2.980602,84.313725
150,WWW,0.093025,22.875817,0.282632,30.718954,0.644867,45.098039
151,WYNN,0.160132,48.366013,0.228410,24.836601,0.440603,37.908497


In [13]:
hqm_score.sort_values('one year return percentile', ascending = False, ignore_index = True)[:10]

,ticker,one year price return,one year return percentile,five year price return,five year return percentile,ten year price return,ten year return percentile
0,KIRK,0.695917,100.0,0.942468,69.281046,2.273326,74.509804
1,TEN,0.618548,99.346405,0.651224,52.287582,1.527101,65.359477
2,CULP,0.553127,98.69281,2.297690,96.732026,7.174029,96.732026
3,MED,0.500701,98.039216,1.123123,76.470588,2.443469,78.431373
4,AMZN,0.492866,97.385621,3.359363,99.346405,18.003294,100.0
5,ESCA,0.440976,96.732026,1.025111,73.856209,0.581413,42.48366
6,RICK,0.419624,96.078431,0.638352,50.980392,1.520094,64.705882
7,LZB,0.408867,95.424837,0.872221,64.705882,1.749440,67.973856
8,DPZ,0.399587,94.771242,2.699532,98.69281,10.065230,98.69281
9,EXPE,0.392397,94.117647,1.724262,89.542484,4.663614,91.503268
